# Agent-based bacteria colony simulation

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
import random
from IPython.display import HTML

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128


In [57]:

class Bacterium:
    def __init__(self, state):  # 0 - empty cell, 1 - young, 2 - ready to divide, 3 - dead
        self.state = state
        self.hunger_level = 0 # if hunger_level == 10 -> dies
        self.can_use_galactose = 0
        self.maturity = 0
        self.is_eating = 0
        
    def get_hungry(self, speed=1):
        self.hunger_level += 1*speed
    
    def learn_to_use_galactose(self):
        if self.hunger_level > 6:
            self.can_use_galactose = 1
            
    def mature(self):
        #if self.hunger_level < 5:
        self.maturity +=1
            
    def check_if_ready_to_divide(self, is_bacteriostatic):
        if self.maturity > 4 and is_bacteriostatic == 0:
            self.state = 2
            
    def after_division(self): # we use it for 2 cells that are created after division
        self.state = 1
        self.hunger_level = 2
        self.can_use_galactose = 0
        self.maturity = 0
    
    def check_if_die(self, is_antibiotic): 
        hunger_level_to_die = 20
        if (self.state == 1 or self.state == 2) and (is_antibiotic == 1 or self.hunger_level == hunger_level_to_die):
            
            if (self.hunger_level == hunger_level_to_die):
                self.state = 3
                
            else:
                if (is_antibiotic == 1):  
                    death = random.choice([0,1])
                    if (death == 1):
                        self.state = 3
                    
             # bacteria is dead
    
    def getColour(self):
        if self.state == 0:   
            return 0
        elif self.state == 1:
            return 255
        elif self.state == 2:
            return 200
        else:
            return 150  # for dead cells
    
    def move(self):
        return

    

In [58]:
class Food:
    def __init__(self, FoodType):
        self.FoodType = FoodType   # 1 - glucose, 2 - galactose, 0 - no food
        if FoodType == 1:
            self.Reserve = 2
        elif FoodType == 2:
            self.Reserve = 4
        else:
            self.Reserve = 0 

    def getColour(self):
        if self.FoodType == 1:   # glucose - light blue
            return "lightblue"
        elif self.FoodType == 2:   # galactose - dark blue
            return "darkblue"

    def check_reserve(self):
        self.Reserve -= 1
        if self.Reserve == 0:
            self.FoodType = 0

    def __repr__(self):
        return str(self.FoodType)
        

In [59]:
class Board:
    
    def __init__(self, xSize, ySize,  bacteriostatic, antibiotic):    
        self.xSize = xSize
        self.ySize = ySize
        self.bacteria_cells = []
        self.food_cells = []
        
        self.is_bacteriostatic = bacteriostatic
        self.is_antibiotic = antibiotic
        
        print('Is bacteriostatic: ', self.is_bacteriostatic)
        print('Is antibiotic: ', self.is_antibiotic)
        print('')
        print('Bacteria legend: ')
        print('Yellow - new bacteria ')
        print('Green - ready to divide ')
        print('Blue - dead bacteria')
        print('')
        print('Food legend: ')
        print('Light blue crosses - glucose (better, more efficient)')
        print('Dark blue crosses - galactose')
             
             
             
        
        
        for i in range(0, xSize):
            b_row = []
            f_row = []
            for j in range(0, ySize):
                b_row.append(Bacterium(0))
                f_row.append(Food(0))
            self.bacteria_cells.append(b_row)
            self.food_cells.append(f_row)
        
        self.randomBoard()

               
    def printBoard(self):
        print('printing the board')
        print(self.cells)
                                                    
    def randomBoard(self):  #TODO - initializing a random board with food
        
        #half glucose, half galactose + random bacteria
    
    
        pivot = ((self.xSize-1)//2)
        for i in range (1,pivot):
            for j in range (1,self.ySize-1, 2):
                    self.food_cells[i][j] = Food(1)
                    
        for i in range (pivot, self.xSize-1, 2):
            for j in range (1,self.ySize-1):
                    self.food_cells[i][j] = Food(2)            
        
        
        fields_mask = np.random.choice([0,1],size=(self.xSize,self.ySize), p=[0.99,0.01])

        for i in range (1,len(fields_mask)-1):
            for j in range (1,len(fields_mask[0])-1):
                if fields_mask[i][j] == 1:
                    self.bacteria_cells[i][j] = Bacterium(1)         
        return

        
    def iteration(self): 
        for i in range(1,len(self.bacteria_cells)-1):
            for j in range(1, len(self.bacteria_cells[0])-1):
                #neigh_cells = [[i-1,j-1], [i-1,j], [i-1,j+1],[i,j-1], [i,j+1], [i+1,j-1],[i+1,j], [i+1,j+1]]
                neigh_cells = [[i-1,j],[i,j-1], [i,j+1], [i+1,j]]
                
               
                
                #for all living bacteria
                if (self.bacteria_cells[i][j].state == 1 or self.bacteria_cells[i][j].state == 2):
                    self.bacteria_cells[i][j].check_if_die(self.is_antibiotic)
                    #self.bacteria_cells[i][j].get_hungry()
                    
                    #if alive, we make it mature and check whether it can reproduce
                    if (self.bacteria_cells[i][j].state == 1 or self.bacteria_cells[i][j].state == 2):
                        #self.bacteria_cells[i][j].mature()
                        self.bacteria_cells[i][j].check_if_ready_to_divide(self.is_bacteriostatic)
                        
                        #reproducing

                        if (self.bacteria_cells[i][j].state == 2): 
                            #koordynat dla nowej bakterii
                            new_x_y = random.choice(neigh_cells)

                            while( new_x_y[0]==0 or new_x_y[0]==len(self.bacteria_cells)-1 or new_x_y[1]==0 or new_x_y[1]==len(self.bacteria_cells[0])-1):
                                new_x_y = random.choice(neigh_cells)
                            
                            new_x = new_x_y[0]
                            new_y = new_x_y[1]
                                
                            if( self.bacteria_cells[new_x][new_y].state==0):
                                self.bacteria_cells[new_x][new_y] = Bacterium(1)
                                self.bacteria_cells[i][j] = Bacterium(1)
                            


                        if self.bacteria_cells[i][j].is_eating == 0:
                            self.bacteria_cells[i][j].get_hungry()
                            move = random.choice(neigh_cells)
                            

                            while( move[0]==0 or move[0]==len(self.bacteria_cells)-1 or move[1]==0 or move[1]==len(self.bacteria_cells[0])-1):
                                move = random.choice(neigh_cells)

                            x,y = i,j  #if they are not moving, the coordinates stay the same

                            if self.bacteria_cells[move[0]][move[1]].state == 0: # they move only if they can - if neighbour cell is free (no other bacterium there)
                                #x,y - new coordinates
                                x,y = move[0],move[1]

                                self.bacteria_cells[x][y].state = self.bacteria_cells[i][j].state
                                self.bacteria_cells[x][y].hunger_level = self.bacteria_cells[i][j].hunger_level
                                self.bacteria_cells[x][y].maturity = self.bacteria_cells[i][j].maturity
                                self.bacteria_cells[i][j].state = 0


                            # after movement we check whether bacterium is on a cell with food
                            # if so - it stays here for a moment
                            # we check it on a new field
                            type_of_food_here = self.food_cells[x][y].FoodType
                            if type_of_food_here == 1:
                                self.bacteria_cells[x][y].is_eating = 2
                                
                            elif type_of_food_here == 2:
                                self.bacteria_cells[x][y].is_eating = 4
                                #self.bacteria_cells[i][j].mature()


                        else:  # wete bakterie zmieniamy dla starego pola, bo te sie nie ruszaja

                            #jesli stoi na komorce z jedzeniem, to skracamy jej stanie o 1
                            
                            if ((self.food_cells[i][j].FoodType)==1):
                                self.bacteria_cells[i][j].mature()
                                self.bacteria_cells[i][j].mature()
                                self.bacteria_cells[i][j].mature()
                                self.bacteria_cells[i][j].hunger_level -= 2
                                
                            elif ((self.food_cells[i][j].FoodType)==2):
                                self.bacteria_cells[i][j].mature()
                                self.bacteria_cells[i][j].hunger_level -= 1
                                
                            self.bacteria_cells[i][j].is_eating -=1

                            if self.bacteria_cells[i][j].is_eating == 0: #jesli po tej akcji jej stan is_eating wynosi 0, to wywalamy to jedzenie z planszy
                                self.food_cells[i][j].FoodType = 0


        
    
    def redraw_bacteria(self):
        # Returns numpy array of colours
        #print('redraws!')
        fig = np.zeros((len(self.bacteria_cells),len(self.bacteria_cells[0])))
        for i in range(0,len(self.bacteria_cells)):
            for j in range(0, len(self.bacteria_cells[0])):
                fig[i][j] = self.bacteria_cells[i][j].getColour()
        #print('fig:', fig)
        return fig
    
    def redraw_food(self):
        # Returns positions of glucose
        glucose_x = []
        glucose_y = []
        galactose_x = []
        galactose_y = []
        for i in range(0,len(self.food_cells)):
            for j in range(0, len(self.food_cells[0])):
                if self.food_cells[i][j].FoodType == 1:
                    glucose_x.append(i)
                    glucose_y.append(j)
                elif self.food_cells[i][j].FoodType == 2:
                    galactose_x.append(i)
                    galactose_y.append(j)
        #indexes = np.where(self.food_cells.FoodType == 1) 
        #print(glucose_x, glucose_y, galactose_x, galactose_y)
        return glucose_x, glucose_y, galactose_x, galactose_y
                
            

In [60]:
def update(frameNum, board, ax):
    if(frameNum == 0):
        board.randomBoard()       
    else:   
        board.iteration()
        ax.clear()
    #im.set_data(board.redraw_bacteria())

    im = ax.imshow(board.redraw_bacteria(),  vmin=0, vmax=255)
    glucose_x, glucose_y, galactose_x, galactose_y = board.redraw_food()
    sct_glucose = ax.scatter(glucose_y, glucose_x, c='lightblue', marker='+', s=150)
    sct_galactose = ax.scatter(galactose_y, galactose_x, c='darkblue', marker='+', s=150)
    return [im, sct_glucose, sct_galactose]

In [61]:
%%capture
# Parameters
steps =40
# Config
fig, ax = plt.subplots(figsize=(10,10))
plt.axis('off')


In [62]:
#Run Simulation
import random 

xSize = 100  #15
ySize= 100
bacteriostatic = 0
antibiotic = 0

board = Board(xSize,ySize, bacteriostatic, antibiotic)



ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [board, ax])
HTML(ani.to_jshtml())


Is bacteriostatic:  0
Is antibiotic:  0

Bacteria legend: 
Yellow - new bacteria 
Green - ready to divide 
Blue - dead bacteria

Food legend: 
Light blue crosses - glucose (better, more efficient)
Dark blue crosses - galactose
